## Libraries

In [12]:
from dotenv import load_dotenv
from os import getenv
import numpy as np
import pandas as pd

load_dotenv()

True

## Global variables

In [ ]:
suppliers = []
rfqs = []
ecns = []

## Classes definition

In [ ]:
#from typing import Any, NewType

class Supplier:
    def __init__(self, name: str, parameters: dict[str, list[float]]):
        self.name = name

        self.µ_price_high_complexity = getenv("AVG_PRICE_LOW_COMPLEXITY")
        self.σ_price_high_complexity = getenv("STDEV_PRICE_LOW_COMPLEXITY")
        self.µ_price_medium_complexity = getenv("AVG_PRICE_MEDIUM_COMPLEXITY")
        self.σ_price_medium_complexity = getenv("STDEV_PRICE_MEDIUM_COMPLEXITY")
        self.µ_price_low_complexity = getenv("AVG_PRICE_HIGH_COMPLEXITY")
        self.σ_price_low_complexity = getenv("STDEV_PRICE_HIGH_COMPLEXITY")

    def __str__(self):
        return self.name

class Part_Number:
    def __init__(self, pn: str, complexity: str):
        self.pn = pn
        self.complexity = complexity

class ECN:
    def __init__(self, ecn_id: str, pn_list: list[Part_Number]):
        self.ecn_id = ecn_id
        self.items = pn_list

    def __str__(self):
        return self.ecn_id

class RFQ:
    def __init__(self, ecn: ECN):
        self.ecn = ecn

In [ ]:
def gen_ecn(qty: int):
    part_kinds = {
        "A": {
            "average": float(getenv("AVG_A_PART_KIND", 0)),
            "stdev": float(getenv("STDEV_A_PART_KIND", 1))
        },
        "B": {
            "average": float(getenv("AVG_B_PART_KIND", 0)),
            "stdev": float(getenv("STDEV_B_PART_KIND", 1))
        },
        "C": {
            "average": float(getenv("AVG_C_PART_KIND", 0)),
            "stdev": float(getenv("STDEV_C_PART_KIND", 1))
        },
        "D": {
            "average": float(getenv("AVG_D_PART_KIND", 0)),
            "stdev": float(getenv("STDEV_D_PART_KIND", 1))
        },
        "E": {
            "average": float(getenv("AVG_E_PART_KIND", 0)),
            "stdev": float(getenv("STDEV_E_PART_KIND", 1))
        },
        "F": {
            "average": float(getenv("AVG_F_PART_KIND", 0)),
            "stdev": float(getenv("STDEV_F_PART_KIND", 1))
        },
        "G": {
            "average": float(getenv("AVG_G_PART_KIND", 0)),
            "stdev": float(getenv("STDEV_G_PART_KIND", 1))
        }
    }

    for key in part_kinds.keys():
        if f"{key}_kind_part_numbers" not in globals():
            globals()[f"{key}_kind_part_numbers"] = []

    for i in range(qty):
        ecn_part_numbers = []

        for key in part_kinds.keys():
            for j in range(round(np.random.normal(loc=part_kinds[key]["average"], scale=part_kinds[key]["stdev"]))):
                pass

A
B
C
D
E
F
G


## ECN definition

In [ ]:
## Part numbers
# Header
A0A000001 = Part_Number(pn="A0A000001", complexity="high")
A0A000002 = Part_Number(pn="A0A000002", complexity="high")
A0A000003 = Part_Number(pn="A0A000003", complexity="high")
A0A000004 = Part_Number(pn="A0A000004", complexity="high")
A0A000005 = Part_Number(pn="A0A000005", complexity="high")
# Distributor
A0B000001 = Part_Number(pn="A0B000001", complexity="medium")
A0B000002 = Part_Number(pn="A0B000002", complexity="medium")
A0B000003 = Part_Number(pn="A0B000003", complexity="medium")
A0B000004 = Part_Number(pn="A0B000004", complexity="medium")
A0B000005 = Part_Number(pn="A0B000005", complexity="medium")


## ECN
ECN0000001 = ECN(ecn_id="ECN0000001", pn_list=[A0A000001, A0A000002, A0B000001, A0B000002])


In [ ]:
df = pd.DataFrame(None, columns=["ECN", "Part number", "Part complexity", "Vendor name", "Price"])
df